<a href="https://colab.research.google.com/github/norojordan/big-data-challenge/blob/main/amazon_books.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
# spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:14 http://security.ubuntu.c

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-23 01:31:55--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-05-23 01:31:55 (10.4 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in Amazon book reviews from S3 into a DataFrame
url = "https://s3.amazonaws.com/nj-hw-bucket/amazon_reviews_us_Books_v1_02.tsv.gz"
spark.sparkContext.addFile(url)

books_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), inferSchema=True, sep='\t',timestampFormat="yyyy-MM-dd")
books_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [5]:
# Count the number of reviews 
print(books_df.count())

3105520


## Drop duplicates and incomplete rows

In [6]:

books_df = books_df.dropDuplicates()
print(books_df.count())

3105520


## Examine the schema

In [9]:
books_df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



## Create a new DataFrame for review info

In [8]:
review_id_info = books_df.select(["customer_id", "product_id", "product_parent", "review_date"])
review_id_info.show(5)

+-----------+----------+--------------+-----------+
|customer_id|product_id|product_parent|review_date|
+-----------+----------+--------------+-----------+
|   53066271|0060194839|     496072800| 2001-05-23|
|   49255792|0375760628|     447916207| 2004-04-13|
|   49394632|0553380966|     199085075| 2003-09-17|
|   34342725|1570719276|     875642269| 2003-06-23|
|   41458372|1561840580|     557383629| 2003-12-23|
+-----------+----------+--------------+-----------+
only showing top 5 rows



## Create a new DataFrame for products info

In [ ]:
products_info=books_df.select(["product_id", "product_title"]).drop_duplicates
products_info.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|0385730586|Sisterhood of the...|
|0811828964|The Bad Girl's Gu...|
|1844161560|Eisenhorn (A Warh...|
|0373836635|Colby Conspiracy ...|
|0262181533|The Psychology of...|
+----------+--------------------+
only showing top 5 rows



## Create a new DataFrame for customers info

In [ ]:
# Total customers count 
customer_count = books_df.select(["customer_id"]).count()
customer_count

3105520

In [11]:
customer_books = books_df.groupBy('customer_id').count()
customer_books.show(5)

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   51163715|    6|
|   12134452|    1|
|   52953209|  110|
|   50926345|    1|
|   17235720|    2|
+-----------+-----+
only showing top 5 rows



In [12]:
# Change column name to match schema
customer_books = customer_books.withColumnRenamed("count", "customer_count") 
customer_books.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   51163715|             6|
|   12134452|             1|
|   52953209|           110|
|   50926345|             1|
|   17235720|             2|
|   50755203|            26|
|   15996792|             1|
|   52222838|             3|
|   20469836|            41|
|   25459059|            34|
|   50093726|           151|
|   12826625|             1|
|   52938787|             2|
|   52507682|             1|
|   50994048|            27|
|   52922003|             6|
|   48313694|             1|
|   50978179|             1|
|   35129585|             1|
|   51893641|             5|
+-----------+--------------+
only showing top 20 rows



## Create a new DataFrame for vine info

In [ ]:
vine_info = books_df.select(["review_id", "star_rating", "helpful_votes", "total_votes","vine"])
vine_info.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RQ58W7SMO911M|          4|            2|          3|   N|
|  RF6IUKMGL8SF|          3|            5|          5|   N|
|R1DOSHH6AI622S|          4|            1|         22|   N|
| RATOTLA3OF70O|          5|            2|          2|   N|
|R1TNWRKIVHVYOV|          4|            0|          2|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



## Write DataFrame to RDS

In [ ]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://mypostgreshwdb.cf9s0qdbvngm.us-east-1.rds.amazonaws.com/my_data_HW_db"
config = {"user":"root",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to pgAdmin table

review_id_info.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

products_info.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

customer_books.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
# Write DataFrame to table

vine_info.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)